In [1]:
import pandas as pds
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [2]:
df = pds.read_csv('titanic/train.csv', sep=",") # load a pandas dataframe from csv in current directory
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Créeons l'attribut "classe". Ici, on s'interesse à la survie des passagers.

In [4]:
y = df["Survived"]
X = df.copy()
del X["Survived"]
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


On doit préprocesser nos données. On peut garder des objets avec des colonnes vides, car les arbres de classification peuvent traiter ce type de données. Il peut aussi traiter les attributs nominatifs.
On enlève l'attribut "Name" et "Embarqued", "PassengerId" et "Ticket" qui ne nous semblent pas utiles.

In [5]:
del X["PassengerId"]
del X["Name"]
del X["Embarked"]
del X["Ticket"]

Apparemment, l'algorithme n'est pas capable de traiter des données en string. On les change.

In [6]:
gender=[]

for element in X['Sex']:
    if element == 'male':
        gender.append(0)
    elif element == "female":
        gender.append(1)
    else:
        gender.append(float('nan'))
            
X["Sex"]=gender

In [7]:
X["Cabin"].unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

In [8]:
cabins=[]

for element in X['Cabin']:
    if type(element)==str:
        letter=element[:1]
        if letter=="A":
            cabins.append(1)
        elif letter=="B":
            cabins.append(2)
        elif letter=="C":
            cabins.append(3)
        elif letter=="D":
            cabins.append(4)
        elif letter=="E":
            cabins.append(5)
        elif letter=="F":
            cabins.append(6)
    else:
        cabins.append(float('nan'))
            
X["Cabin"]=cabins

ValueError: Length of values (886) does not match length of index (891)

Créons notre arbre de décision

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X,y)

Utilisons notre arbre pour prédire le taux de survie du dataframe test

In [ ]:
df_test = pds.read_csv('titanic/test.csv', sep=",") # load a pandas dataframe from csv in current directory
df_test.dtypes